In [1]:
import pandas as pd
train= pd.read_csv("processed/train.csv")
test= pd.read_csv("processed/test.csv")

In [2]:
y_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[y_classes].values
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint



Using TensorFlow backend.


In [3]:
train.comment_text

0         explanation edits made username hardcore metal...
1         daww match background colour seemingly stuck t...
2         hey man really not trying edit war guy constan...
3         cannot make real suggestion improvement wonder...
4                             sir hero chance remember page
5                    congratulation well use tool well talk
6                               cocksucker piss around work
7         vandalism matt shirvington article ha reverted...
8         sorry word nonsense wa offensive anyway not in...
9                      alignment subject contrary dulithgow
10        fair use rationale image wonju jpg thanks uplo...
11                         bbq man let discus itmaybe phone
12        hey talk exclusive group wp taliban good destr...
13        start throwing accusation warning let review e...
14        oh girl started argument stuck nose doe not be...
15        juelz santanas age 2002 juelz santana wa 18 ye...
16              bye not look come think 

In [4]:
test[test.comment_text.isnull()==True]

,id,comment_text
34,000d4f120d5a7303,NaN
114,002c586b0af3792c,NaN
119,003044a2c35274b6,NaN
404,00ad0323d9b31f21,NaN
419,00b568d3b0f61a37,NaN
506,00d2aca8d65aa590,NaN
578,00f496156c7ace6b,NaN
582,00f5061275635c3a,NaN
662,0118f7ea375be714,NaN
674,011f5e640910e527,NaN


In [5]:
raw_test= pd.read_csv("data/test.csv")

In [6]:
raw_test.iloc[2072]

id                                               03952d035c3abdb4
comment_text    ::Γεια σου. Είνε πάρα πολύ δουλειά! Έκανες πολ...
Name: 2072, dtype: object

In [7]:
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [8]:

tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list_sentences_train)

In [11]:
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=100)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=100)
def get_model():
    embed_size = 128
    inp = Input(shape=(100, ))
    x = Embedding(20000, embed_size)(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.2)(x)
    #x = Dense(50, activation="relu")(x)
    #x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
model = get_model()
batch_size = 64
model.fit(X_t, y, epochs=1, batch_size=batch_size, 
          shuffle=True, validation_split=0.1, verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/1
 96896/143613 [===================>..........] - ETA: 2:19 - loss: 0.0823 - acc: 0.9751 